# RNN を使ったテキスト分類

In [4]:
# !pip install -q tf-nightly
# import tensorflow_datasets as tfds
# !pip3 list | grep tensorflow
# import tensorflow as tf

# !pip3 install sentencepiece
import sentencepiece as spm

import json
import os
import pandas as pd

## Data


### load

In [5]:
sbj_names = [
    # ['Eigo']
    ['Suugaku']
    # ['Eigo', 'Suugaku']
][0]
elmnt_name = 'question'
attr_name = 'knowledge_type'  # 'answer_type'   #

df = None
for sbj in sbj_names:
    print(sbj)
    attr_csv_path = f'../{sbj}_{attr_name}_ds.tsv'
    df_tmp = pd.read_csv(attr_csv_path, delimiter='\t')
    df = pd.concat([df, df_tmp])

df = df.reset_index(drop=True)
df = df.dropna()  # nan を削除
df

Eigo
Suugaku


,Unnamed: 0,knowledge_type,<instruction/>,contents
0,0,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問１</label> <ansColumn id=""A1"">1</ansC..."
1,1,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問２</label> <ansColumn id=""A2"">2</ansC..."
2,2,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問３</label> <ansColumn id=""A3"">3</ansC..."
3,3,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問４</label> <ansColumn id=""A4"">4</ansC..."
4,4,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問５</label> <ansColumn id=""A5"">5</ansC..."
...,...,...,...,...
3028,393,MATH_IIB_VECTOR,三つのベクトル，，について …………………………① …………………………②ア，イに当てはまる...,<label>(2)</label> <instruction><formula />...
3029,394,MATH_IIB_VECTOR,により，三角形ABCは正三角形である。以下，4点A，B，C，Dが，正四面体の四つの頂点になる...,<label>(3)</label> <instruction><formula>(x...
3030,395,"IC_O,IC_T,MATH_IIB_STATISTICS",0＜p＜1とする。袋の中に白球がp，赤球が1-pの割合で，全部でm個入っているものとする...,<label>(1)</label> <instruction><formula />...
3031,396,"IC_O,IC_T,MATH_IIB_STATISTICS",とする。この袋の中から1個の球を取り出し袋の中へ戻すという試行を4回繰り返すとき，白球の出る...,<label>(2)</label> <instruction><formula>m=...


### Tokenize
SentencePiece を使用。
- タグ あり／なし

In [6]:
from sklearn.model_selection import train_test_split

train_examples, test_examples = train_test_split(
                                    df, test_size=0.4, random_state=0)
train_examples.head(5)

,Unnamed: 0,knowledge_type,<instruction/>,contents
270,270,R_QA,１～４)の空欄( 42 ～ 45 )に入れるのに最も適当なものを，それぞれ以下の①～④のうち...,"<label>問１</label><data id=""D29"" type=""text"">Wh..."
2351,2351,"DIS_W,R_ENT",次の文章を読み，下の問い（Ａ・Ｂ）に答えよ。なお，文章の左にある(1)～(6)は段落の番...,"<label>問５</label><data id=""D48"" type=""text""> T..."
1431,1431,DIS_S,次の問い(問１・問２)において，文章の 28 ・ 29 に入れる三つの文が，順不同で以下のA...,"<label>問１</label> <data id=""D22"" type=""text..."
475,475,"DIS_S, IC_O",(配点 16),"<label>問３</label><ansColumn id=""A44"">44</ansCo..."
532,532,"DIS_S, R_ENT",(配点 24),"<label>問３</label><data id=""D43"" type=""text"">Th..."


In [7]:
df_tmp = df[['<instruction/>', 'contents']]
df_tmp['<instruction/>'][6]

'次の一連の文章(問１・２)の中の①～③および④～⑥には，それぞれ強く発音されるべき語が一つずつある。その語を選べ。'

In [8]:
m_dir = '_logs/SentencePiece'
os.makedirs(m_dir, exist_ok=True)
df.to_csv(f'{m_dir}/tmp.txt', sep='\t')

# arg_str = '--input={m_dir}/tmp.txt --model_prefix={m_dir}/m_user ' + '--user_defined_symbols=<sep>,<cls>' + ',<ansColumn/>,<label>' + ' --vocab_size=2000'
# spm.SentencePieceTrainer.train(arg_str)

spm.SentencePieceTrainer.train(f'--input={m_dir}/tmp.txt --model_prefix={m_dir}/m  --user_defined_symbols=<sep>,<cls>,<pad>   --vocab_size=2000')
sp = spm.SentencePieceProcessor()  # model_file='SentencePiece/test_model.model'

sp.load(f'{m_dir}/m.model')

True

In [9]:
# encode: text => id
tokenized_tokens =  sp.encode_as_pieces('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
print(tokenized_tokens)

tokenized_ids = sp.encode_as_ids('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
print(tokenized_ids)

decoded_text = sp.decode(tokenized_ids)
print(decoded_text)

['▁次の問い', '(', '問', '1～3)', 'の会話の', '▁17', '▁～', '▁19', '▁に入れるのに最も適当な', 'ものを', ',', 'それぞれ以下の', '1～4', 'のうちから一つずつ選', 'べ', '。']
[70, 32, 29, 188, 609, 197, 79, 218, 169, 64, 20, 93, 59, 66, 58, 35]
次の問い(問1～3)の会話の 17 ～ 19 に入れるのに最も適当なものを,それぞれ以下の1～4のうちから一つずつ選べ。


In [10]:
example_content = df_tmp['contents'][20]
print(example_content, sp.encode_as_pieces(example_content))

  <label>問14</label>  <data id="D20" type="text"> <ansColumn id="A21">21</ansColumn></data>  <choices anscol="A21" comment=""> <choice><cNum>①</cNum> Yes, thank you.  </choice> <choice><cNum>②</cNum> Yes, that's O.K.  </choice> <choice><cNum>③</cNum> Not at all.  </choice> <choice><cNum>④</cNum> I'd be glad to.  </choice> </choices> ['▁<', 'label', '>', '問', '14', '</', 'label', '>', '▁<', 'data', '▁id', '=', '"', 'D', '20', '"', '▁type', '=', '"', 'text', '"', '>', '▁<', 'ansColumn', '▁id', '=', '"', 'A', '21', '"', '>21</', 'ansColumn', '></', 'data', '>', '▁<', 'choices', '▁ans', 'col', '=', '"', 'A', '21', '"', '▁comment', '=""', '>', '▁<', 'choice', '><', 'c', 'Num', '>1</', 'c', 'Num', '>', '▁Yes', ',', '▁than', 'k', '▁you', '.', '▁</', 'choice', '>', '▁<', 'choice', '><', 'c', 'Num', '>2</', 'c', 'Num', '>', '▁Yes', ',', '▁that', "'", 's', '▁', 'O', '.', 'K', '.', '▁</', 'choice', '>', '▁<', 'choice', '><', 'c', 'Num', '>3</', 'c', 'Num', '>', '▁No', 't', '▁at', '▁all', '.', '▁<

In [11]:
# for index in encoded_string:
#   print('{} ----> {}'.format(index, encoder.decode([index])))

## Train 用データの準備

In [12]:
word2index = {}
# 系列を揃えるためのパディング文字列<pad>を追加
# パディング文字列のIDは0とする
word2index.update({"<pad>":0})

for inst, cont in zip(df['<instruction/>'], df['contents']):
#     try:
    tokens = sp.encode_as_pieces(inst + cont)
    for word in tokens:
            if word in word2index: continue
            word2index[word] = len(word2index)
#     except TypeError:
#         print(f'[Error] <instruction/> が nan です。')
#         print(f'    inst : {inst}')
#         print(f'    cont : {cont}')

print("vocab size : ", len(word2index))


vocab size :  2163


In [13]:
## set_dict から自動抽出する！
# attr_name = 'knowledge_type'  # 'answer_type'

categories = set()
for sbj in sbj_names:
    with open(f'../class_set/{sbj}-{elmnt_name}-{attr_name}.json') as f:
        categories |= set(json.load(f))   # sbj_names = ['Eigo', ]

# print(categories)

categories = list(categories)
categories.sort()    # 入れないと、クラス番号が変わってしまい、再現実験ができないので注意？
print(categories)
print(len(categories))

['DIC_O', 'DIC_O, DIS_S, R_ENT', 'DIC_O, DIS_W', 'DIC_O, DIS_W, GK', 'DIC_O, EG', 'DIC_O, EG, DIS_W', 'DIC_O, EG, GK', 'DIC_O, GK', 'DIC_O, Other', 'DIC_O, SEL', 'DIC_O,DIS_W', 'DIC_O,GK', 'DIS_C, GK', 'DIS_O', 'DIS_O, DIS_W', 'DIS_O, DIS_W, GK', 'DIS_O, GK', 'DIS_O, Other', 'DIS_S', 'DIS_S, IC_O', 'DIS_S, R_ENT', 'DIS_S, R_ENT, IC_G', 'DIS_S, R_ENT, IC_M', 'DIS_S, R_ENT, IC_O', 'DIS_S, R_ENT, IC_T', 'DIS_S, R_ENT, R_SUM', 'DIS_S,R_ENT', 'DIS_S,R_ENT,R_SUM', 'DIS_S,R_QA', 'DIS_W', 'DIS_W, GK', 'DIS_W, R_ENT', 'DIS_W, R_ENT, IC_G', 'DIS_W, R_ENT, IC_G, IC_P', 'DIS_W, R_ENT, IC_M', 'DIS_W, R_ENT, IC_O', 'DIS_W, R_ENT, IC_O, IC_P', 'DIS_W, R_ENT, IC_P', 'DIS_W, R_ENT, IC_P, IC_T', 'DIS_W, R_ENT, IC_T', 'DIS_W, R_ENT, R_SUM', 'DIS_W, R_ENT, R_SUM, IC_O', 'DIS_W, R_QA', 'DIS_W,DIC_O', 'DIS_W,EG', 'DIS_W,GK', 'DIS_W,IC_P', 'DIS_W,R_ENT', 'DIS_W,R_ENT,IC_P', 'DIS_W,R_QA', 'DIS_W,R_SUM', 'DIS_W,R_SUM,IC_T', 'EG', 'EG, ', 'EG, DIS_S', 'EG, DIS_W', 'EG, DIS_W, GK', 'EG, GK', 'EG,DIS_W', 'EG,SEL'

In [20]:
## 系列の長さを揃えてバッチでまとめる
from sklearn.model_selection import train_test_split
import numpy as np
import random
from tqdm import tqdm

cat2index = {}
for cat in categories:
    if cat in cat2index: continue
    cat2index[cat] = len(cat2index)

def sentence2index(sentence):
    tokens = sp.encode_as_pieces(sentence)
    # print(tokens)
    return [word2index[w] for w in tokens]

def category2index(cat):
    return cat2index[cat]

index_datasets_c_xml_tmp = []
index_datasets_category = []

# 系列の長さの最大値を取得。この長さに他の系列の長さをあわせる
max_len = 0
for inst, cont, category in tqdm(zip(df['<instruction/>'], df['contents'], df[attr_name])):
    index_c_xml = sentence2index(inst + cont)
    index_category = category2index(category)
    index_datasets_c_xml_tmp.append(index_c_xml)
    index_datasets_category.append(index_category)
    if max_len < len(index_c_xml):
        max_len = len(index_c_xml)
        # if max_len > 10000:
        #     print(inst, cont)

# 系列の長さを揃えるために短い系列にパディングを追加
index_datasets_c_xml = []
for c_xml in tqdm(index_datasets_c_xml_tmp):
    # パディング作成
    padd = [0] * (max_len - len(c_xml))
    # 後ろパディングだと正しく学習できなかったので、前パディング
    c_xml = padd + c_xml # 前パディング
    # c_xml = c_xml + padd # 後ろパディング
#     print(len(c_xml))
    index_datasets_c_xml.append(c_xml)

x_train, x_valid, y_train, y_valid = train_test_split(index_datasets_c_xml, index_datasets_category, train_size=0.7)
x_train = np.array(x_train)
y_train = np.array(y_train)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)
print(x_train[:5])

3016it [00:01, 1722.17it/s]
100%|██████████| 3016/3016 [00:00<00:00, 5548.75it/s]


AttributeError: 'list' object has no attribute 'head'

In [17]:
from sklearn.preprocessing import StandardScaler
import torch

# 特徴量の標準化
# scaler = StandardScaler()
# scaler.fit(x_train)
# x_train = scaler.transform(x_train)
# x_valid = scaler.transform(x_valid)

# Tensor型に変換
# 学習に入れるときはfloat型 or long型になっている必要があるのここで変換してしまう
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)
x_valid = torch.from_numpy(x_valid)
y_valid = torch.from_numpy(y_valid)

print('x_train : ', x_train.shape)
print('y_train : ', y_train.shape)
print('x_valid : ', x_valid.shape)
print('y_valid : ', y_valid.shape)
print(x_train[:5])

NameError: name 'y_train' is not defined

In [ ]:
###  Dataset  ###
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)

# 動作確認
# indexを指定すればデータを取り出すことができます。
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

## Model

In [ ]:
import torch
# import torch.nn.functional as F
import torch.optim as optim

%load_ext autoreload
from model_abc.LSTM_text_classify_model import (
    LSTM_TextClassifier_ptModel
)
%autoreload

# GPUを使うために必要
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# 単語の埋め込み次元数上げた。精度がそこそこアップ！ハイパーパラメータのチューニング大事。
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)

## モデルの保存場所を準備する。
import datetime
dt_now = datetime.datetime.now()
save_m_dir = os.path.join('_logs', dt_now.strftime('%Y-%m-%d_%Hh%Mm%Ss'))

model = LSTM_TextClassifier_ptModel(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE,
                                        save_m_dir, save_model_name='LSTM_classifier_.pth').to(device)

## Experiment Train

In [19]:
import torch.nn as nn
from base_ExpTrain import Batch_ExpTrain


lr=0.001
epochs=1500
batch_size=300
early_stopping=500

exp_batch_train = Batch_ExpTrain(train_dataset, valid_dataset, device)
criterion = nn.NLLLoss()  # ignore_index=PAD_token
optimizer = optim.Adam(model.parameters(), lr=lr)
max_valid_acc = exp_batch_train.exec(
                            model, criterion, optimizer,
                            epochs=epochs, batch_size=batch_size, early_stopping=early_stopping )
                            # teacher_forcing=0.5, early_stopping=5)

print(max_valid_acc)
print("done.")

NameError: name 'train_dataset' is not defined

In [ ]:
## 実験パラメータのメモを保存
import json
with open(os.path.join(save_m_dir, 'model_params.json'), 'w') as param_f:
    json.dump({
        'Model' : {
            'EMBEDDING_DIM': EMBEDDING_DIM,
            'HIDDEN_DIM' : HIDDEN_DIM,
            'VOCAB_SIZE' : VOCAB_SIZE,
            'TAG_SIZE':TAG_SIZE,
        },
        'Experiment' : {
            'XML条件' : {
                'sbj_names': sbj_names,
                'elmnt_name': elmnt_name,
                'attr_name': attr_name
            },
           '実験設定' : {
                'lr':lr,
                'epochs':epochs,
                'batch_size':batch_size,
                'early_stopping':early_stopping
            },
            '実験結果' : {
                'lr':lr,
                'epochs':epochs,
                'batch_size':batch_size,
                'early_stopping':early_stopping
            }
        },
    }, param_f)

___

## Experiment Acc 計算

In [ ]:
test_num = len(x_valid)
valid_acc = 0
valid_dataloader = DataLoader(valid_dataset, batch_size=1, shuffle=False)


with torch.no_grad():
    for idx, (X_batch, Y_batch) in enumerate(valid_dataloader):
        valid_loss = 0
        valid_loss, pred_batch_arr = model.predict(X_batch, Y_batch, criterion, self.device)
        # acc を計算する。
        _, pred_batch = torch.max(pred_batch_arr, 1)
        # acc を計算する。
        for j, ans in enumerate(Y_batch):
            # print(pred_batch[j].item(), ans.item())
            if pred_batch[j].item() == ans.item():
                valid_acc += 1
#             else:
#                 print(predicts[j].item(), ans.item())
        total_count += Y_batch.size(0)
    valid_acc /= total_count
print("[Info] acc : ", valid_acc, "loss : ", valid_loss)
# predict :  0.6967916854948034

100%|██████████| 10/10 [01:54<00:00, 11.49s/it]

predict :  0.02983425414364641


___